In [1]:
import nibabel as nib
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import json

In [9]:
ho_sub_path = os.path.join('harvard-oxford', 'HarvardOxford-sub-maxprob-thr0-1mm.nii.gz')
ho_cor_path = os.path.join('harvard-oxford', 'HarvardOxford-cortl-maxprob-thr0-1mm.nii.gz')
aal_path = 'aal_combined.nii.gz'

In [10]:
ho_cor = nib.load(ho_cor_path)
ho_sub = nib.load(ho_sub_path) 
aal = nib.load(aal_path)
affine = ho_cor.affine

In [11]:
template = nib.load('MNI152_T1_1mm_brain.nii.gz').get_fdata()
ho_cor_v = ho_cor.get_fdata()
ho_sub_v = ho_sub.get_fdata() 
aal_v = aal.get_fdata()

In [23]:
def get_atlas(atlas_name, atlases=[ho_cor_v, ho_sub_v, aal_v]):
    if atlas_name == 'harvard-cortical':
        return ho_cor_v
    if atlas_name == 'harvard-sub':
        return ho_sub_v
    if atlas_name == 'aal':
        return aal_v
    if atlas_name == "aal combined":
        return aal_v

In [24]:
def obtain_region_mask(region_group, fnum, template=template):
    mask = np.zeros(template.shape)
    for r in region_group:
        atlas = get_atlas(r['atlas'])
        num = r['region_number']
        mask+=((atlas==num).astype(np.int16))
    #print(mask.sum())
    return fnum*(mask>0).astype(np.int16)
    

In [25]:
def region_difference(groupA, groupB, fnum):
    A = obtain_region_mask(groupA, 9999)
    B = obtain_region_mask(groupB, fnum)
    dif = np.abs(A-B)
    return fnum*((dif==fnum).astype(np.int16))

In [26]:
with open('aal_ho_difference_combined.json') as f:
    regions_data = json.load(f)

In [27]:
sample = regions_data['left amygdala']
print(regions_data["left amygdala"])
((region_difference(sample['A'], sample['B'], sample['number']))==sample['number']).sum()

{'number': 1, 'A': [{'region_name': 'left amygdala', 'region_number': 10, 'atlas': 'harvard-sub'}], 'B': [{'region_name': 'left amygdala', 'region_number': 1, 'atlas': 'aal combined'}]}


385

In [28]:
regions = list(regions_data.keys())
fnums = []
new_vol = np.zeros(template.shape)
for r in tqdm(regions):
    RA = regions_data[r]['A']
    RB = regions_data[r]['B']
    fnum = regions_data[r]['number']
    fnums.append(fnum)
    dif = region_difference(RA, RB, fnum)
    new_vol+=dif
pd.DataFrame({'name':regions, 'index':fnums}).to_csv('differences_ho_aal_combined.csv', index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:05<00:00,  2.29it/s]


In [29]:
dif_at = nib.Nifti1Image(new_vol, affine)
nib.save(dif_at, 'differences_ho_aal_combined.nii.gz')